## Rename tables

In [6]:
import sqlite3

db_path = '../data/fpl.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Find all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

# Iterate through the tables
for table in tables:
    old_name = table[0]
    # Remove prefix number and underscore, then convert to lowercase
    new_name = old_name.lstrip('0123456789_').lower()
    if old_name[:2] == "01":
        new_name = f"raw_{new_name}"

    # Rename the table
    if old_name != new_name:
        print(f"{old_name}-->{new_name}")
        if old_name == new_name.upper():
            cursor.execute(f"ALTER TABLE \"{old_name}\" RENAME TO \"01_{old_name}\"")
            cursor.execute(f"ALTER TABLE \"01_{old_name}\" RENAME TO {new_name}")
        else:
            cursor.execute(f"ALTER TABLE \"{old_name}\" RENAME TO {new_name}")


# Commit the changes and close the connection
conn.commit()
conn.close()

## Rename columns

In [13]:
db_path = '../data/fpl.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Find all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

# Iterate through the tables
for table in tables:
    table_name = table[0]

    # Get column names for the table
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()

    # Create a temporary table with lowercase column names
    new_columns = []
    for col in columns:
        name = col[1]
        _type = col[2]
        name = name.lower()
        if name == "index":
            name = "id"
        new_columns.append(f"{name} {_type}")
    old_columns = [col[1] if col[1] != "index" else "'index'" for col in columns]
    new_columns_without_type = [col.split()[0] for col in new_columns]

    if old_columns!=new_columns_without_type:
        new_columns_str = ', '.join(new_columns)
        cursor.execute(f"CREATE TABLE temp_table ({new_columns_str})")

        # Copy data from the original table to the temporary table
        old_columns_str = ', '.join(old_columns)
        cursor.execute(f"INSERT INTO temp_table SELECT {old_columns_str} FROM {table_name}")

        # # Drop the original table
        cursor.execute(f"DROP TABLE {table_name}")

        # # Rename the temporary table to the original table name
        cursor.execute(f"ALTER TABLE temp_table RENAME TO {table_name}")

        print("Renamed all columns in table", table_name)

# Commit the changes and close the connection
conn.commit()
conn.close()